<a href="https://colab.research.google.com/github/RyanChen12035/w266-NLP/blob/main/w266_final_project_model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
1. Use GPT2-medium, Decoder based, pretrained with EN text and has 24 layers of transformer layers. On top of it is a projection layer with shape(768, 50257) and softmax layer
2. It was pretrained by EN text, so basically it can not handle EN to CN task without fine-tuning
3. Need a token that incorperate EN and CN, the embedding size and dictionary size are (768, 119,547)
4. Fine-tune, may need prompt + EN,CN pair. Detailed training process needs to be further clearfy.
5. The FFN consists of two linear transformations with weights of shapes (768, 3072) and (3072, 768)
6. Only need a look-ahead mask before the first layer of transformer. For following layers, input would be pass down one by one without seeing future information.
7. No CLS token in decoder based model, so after 10 runs of decoding process (assuming we have 10 input length, pass one token and previous input for
   next layer each time), we would have (batch, sequence, embedding dimensions) as the shape of each transformer layer. We can take mean pooling or max pooling
   to have the representation of each layers of transformer.
8. The shape of representation would be (1,768) and we can put it into the comparison of neurons in FFN.

"""

In [ ]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output
from transformers import BertTokenizer, GPT2LMHeadModel
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout, Conv1D, GlobalMaxPooling1D, Concatenate, Activation
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
from transformers import BertTokenizer, TFBertModel
from transformers import logging
logging.set_verbosity_error()
import sklearn as sk
import os
from nltk.data import find
import matplotlib.pyplot as plt
import re

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
!nvidia-smi -L

In [ ]:
#check out the data source we have
tmp_builder = tfds.builder("wmt19_translate/zh-en")
pprint(tmp_builder.subsets)

In [ ]:
#download data by tfds.builder
config = tfds.translate.wmt.WmtConfig(
  version=tfds.core.Version('0.0.3'),
  language_pair=("zh", "en"),
  subsets={
    tfds.Split.TRAIN: ["newscommentary_v14"]
  }
)
builder = tfds.builder("wmt_translate", config=config)
builder.download_and_prepare(download_dir=download_dir)
clear_output()

In [ ]:
#set builder to dataset(data pipeline type), split it into training, validation, testing
examples = builder.as_dataset(split=['train[:20%]','train[20%:21%]','train[21%:]'], as_supervised=True)

In [ ]:
#leave the testing examples this time.
train_examples, val_examples, _ = examples
print(train_examples)
print(val_examples)

In [ ]:
sample_examples = []
num_samples = 10

for en_t, zh_t in train_examples.take(num_samples):
  en = en_t.numpy().decode("utf-8")
  zh = zh_t.numpy().decode("utf-8")

  print(en)
  print(zh)
  print('-' * 10)


  sample_examples.append((en, zh))

In [ ]:

# Initialize a tokenizer for English and Chinese
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

# Initialize the GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2-medium")